In [44]:
import pandas as pd
import numpy as np
import re 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, HashingVectorizer

In [143]:
path = '/home/victor/Documents/Expectation_data_generation/src/meals_collection/df_final_7000_with_ingredients_fixed.csv'
df_recipes = pd.read_csv(path, index_col=0, sep='|')

In [144]:
df_recipes.head(2)

,title,raw_text,cultural_restriction,calories,allergies,recipeId,ingredients,preparation,carbs,fat,fiber,protein,taste,cooking_style,meal_type,prep_time,cuisine,price,ingredients_list
0,Fruit Salad,"1. Fruit Salad: 70 calories per portion, 4 por...",vegan,70.0,contains fruits only,food_0,ingredients:\n- 1 apple\n- 1 banana\n- 1 orang...,\n1. wash and cut all the fruits into bite-siz...,223.0,2.0,0.0,15.0,sweet,mixed,fruit-based,NaN,Fruit Salad - Mixed,2.0,"apple, banana, orange, grape, strawberry, pine..."
1,Vegan Omelette,"5. Vegan Omelette: 300 calories per portion, 1...",vegan,300.0,contains chickpea flour,food_4,ingredients:\n- 1 cup chickpea flour\n- 1 cup ...,"\n1. in a mixing bowl, whisk together the chic...",100.0,20.5,6.0,15.0,Savory,sauteed,veggie,20.0,Vegan Omelette - Vegan,2.0,"chickpea, flour, water, yeast, turmeric, garli..."


In [145]:
# check which columns have empty values 
empty_cols = {}
for col in df_recipes.columns:
    num_empty = sum(df_recipes[col].isna())
    if num_empty > 0:
        empty_cols[col] = num_empty
    

In [146]:
sorted(empty_cols.items(), key=lambda x:x[1])

[('carbs', 62),
 ('protein', 120),
 ('fat', 180),
 ('fiber', 735),
 ('meal_type', 756),
 ('cooking_style', 779),
 ('cultural_restriction', 805),
 ('prep_time', 1709),
 ('allergies', 4938)]

In [147]:
# Checking ingredients 
# mask = df_recipes['ingredients'].str.contains(' unfortunately ')
# print(sum(mask))
mask = df_recipes['carbs'].isna()
print(sum(mask))

62


In [148]:
df_to_fix = df_recipes.loc[mask, :]

In [150]:
print(df_to_fix['ingredients'].tolist())

['ingredients:\n- 1 ½ cups all-purpose flour (gluten-free flour can be substituted if desired)\n- 2 tablespoons almond meal\n- 2 teaspoons baking powder\n- 1/4 teaspoon salt\n- 1/4 cup cold vegan butter, cut into small pieces\n- 3 tablespoons maple syrup or agave nectar\n- 1/2 cup unsweetened almond milk\n- 1 teaspoon vanilla extract\n- 1/2 cup fresh or frozen raspberries\n- 2 tablespoons sliced almonds', 'ingredients:\n- 1 cup pumpkin puree\n- 1 3/4 cups plant-based milk\n- 2 tablespoons maple syrup\n- 1 teaspoon vanilla extract\n- 2 cups all-purpose flour\n- 2 tablespoons sugar\n- 2 teaspoons baking powder\n- 1 teaspoon baking soda\n- 1 teaspoon ground cinnamon\n- 1/2 teaspoon ground ginger\n- 1/4 teaspoon ground nutmeg\n- 1/4 teaspoon ground cloves\n- 1/4 teaspoon salt\n\n', "sure! here's a recipe for sweet potato chips:\n\ningredients:\n- 2 medium sweet potatoes\n- salt, to taste\n- olive oil\n\n", 'ingredients:\n- 1 large cucumber\n- 8 ounces vegan cream cheese\n- 1 tablespoon fre

In [141]:
df_recipes.loc[mask, 'cuisine'] = ['Snack', 'Dessert', 'Vegan', 'Italian']

In [97]:
def process_ingredients(raw_ingredients:str):
    ingredients = raw_ingredients.split('\n\n')
    ingredients_list = []
    for i in range(1, len(ingredients)):
        if len(ingredients[i]) > 0:
            ingredients_list.append(ingredients[i])
    #print(len(ingredients_list))
    return "\n".join(ingredients_list)

In [98]:
text_test = """
unfortunately, as a language model ai, i don't have access to a specific recipe database or the ability to provide specific nutritional information like calories. however, i can give you a general recipe for vegan cinnamon rolls, and you can use online tools or apps to calculate the calories based on your specific ingredients and serving sizes. here's a simple recipe:

ingredients:
- 2 ½ cups all-purpose flour
- 2 tsp baking powder
- ½ tsp salt
- 1 cup unsweetened almond milk (or any non-dairy milk)
- ¼ cup melted coconut oil (or any vegetable oil)
- ¼ cup maple syrup (or any sweetener of choice)
- 1 tsp vanilla extract

for the cinnamon filling:
- ½ cup brown sugar
- 2 tbsp ground cinnamon
- ¼ cup melted vegan butter (or any vegetable spread)

for the glaze:
- 1 cup powdered sugar
- 1-2 tbsp non-dairy milk
- ½ tsp vanilla extract
"""
res = process_ingredients(text_test)
print(res)

ingredients:
- 2 ½ cups all-purpose flour
- 2 tsp baking powder
- ½ tsp salt
- 1 cup unsweetened almond milk (or any non-dairy milk)
- ¼ cup melted coconut oil (or any vegetable oil)
- ¼ cup maple syrup (or any sweetener of choice)
- 1 tsp vanilla extract
for the cinnamon filling:
- ½ cup brown sugar
- 2 tbsp ground cinnamon
- ¼ cup melted vegan butter (or any vegetable spread)
for the glaze:
- 1 cup powdered sugar
- 1-2 tbsp non-dairy milk
- ½ tsp vanilla extract



In [99]:
df_recipes.loc[mask, 'ingredients'] = df_recipes.loc[mask, 'ingredients'].apply(lambda x: process_ingredients(x))

In [100]:
sum(mask)

1

In [82]:
df_recipes.loc[mask, 'ingredients']

965     ingredients:\n- 8 oz rice noodles\n- 4 cups ve...
3013    ingredients:\n- 2 cups all-purpose flour\n- 1/...
3053    ingredients:\n- 2 cups dried fava beans\n- 4 c...
4149    ingredients:\n- 14 oz (400g) firm tofu, cubed\...
4910    ingredients:\n- fresh berries (strawberries, b...
Name: ingredients, dtype: object

In [31]:
# checking nutritional values 
mask = df_recipes['carbs'].isna()
print(sum(mask))

62


In [14]:
df_to_fix = df_recipes.loc[mask, :]

In [15]:
df_to_fix[['title','ingredients']].to_json()

'{"title":{"167":"Vegan Raspberry Almond Scones","378":"Vegan Pumpkin Spice Pancakes","590":"Sweet Potato Chips ","627":"Cucumber Rolls with Vegan Cream Cheese ","1762":"Coconut Yogurt Parfait with Berries","1781":"Mozzarella Sticks","1931":"Fresh Squeezed Orange Juice","1959":"Homemade Veggie Pizza","2024":"Caprese Skewers with Balsamic Glaze","2051":"Caprese salad skewers","2054":"Tomato and mozzarella skewers","2103":"Stuffed dates with almond butter","2149":"Greek Feta and Olive Bread","2192":"Veggie Chips","2201":"Mini Pizzas","2252":"Yogurt Covered Strawberries","2306":"Tomato and Mozzarella Skewers","2427":"Caprese salad","3013":"Turkish G\\u00f6zleme","3053":"Classical Arab Ful Medames","3616":"Mini ham and cheese sandwiches","3665":"Mini Pita Pizzas","3697":"Fried Plantain","4148":"Herb-roasted turkey breast with sweet potatoes","4162":"Greek stuffed chicken breast with roasted potatoes","4203":"Honey glazed sesame chicken with brown rice","4546":"Cheese and Tomato Toastie","4

In [142]:
df_recipes.to_csv('/home/victor/Documents/Expectation_data_generation/src/meals_collection/df_final_7000_with_ingredients_fixed.csv', 
                  sep='|')

In [17]:
df_to_fix = df_recipes.loc[mask, :]

In [18]:
df_recipes.loc[mask, "ingredients"] = df_to_fix["ingredients"].apply(lambda x: x.split("\n\n")[-1]).tolist()

In [19]:
# fixing columns 
mask = df_recipes['carbs'].isna()
df_to_fix = df_recipes.loc[mask, :]

In [20]:
df_to_fix['ingredients'].tolist()

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '9. close the waffle iron and cook according to your waffle iron’s ',
 "9. **cook the waffles**. pour an appropriate amount of batter onto the preheated waffle iron (the amount will depend on the size of your waffle iron; usually, it's about 1/2 cup to 1 cup). spread the batter close to the edges, and close the lid. cook according to the waffle iron's ",
 'loin pork chops (1 per person \n  1/2-inch thick) \n  trimmed of fat \n  apples \n  halved and cored (not peeled) \n  onions \n  sliced 1/2-inch thick \n  salt and pepper to taste',
 '1 box yellow cake mix \n  4 eggs \n  1 can mandarin oranges \n  1/2 c. oil',
 '1 large pkg. Oreo cookies \n  1 1/2 gal. Blue Bell peppermint ice cream \n  1 stick butter',
 '1 egg \n  1 Ziploc Baggie',
 '1 bag boneless skinless thawed chicken breasts \n  1 package provol

In [21]:
df_to_fix['title'].tolist()

['Vegan Raspberry Almond Scones',
 'Vegan Pumpkin Spice Pancakes',
 'Sweet Potato Chips ',
 'Cucumber Rolls with Vegan Cream Cheese ',
 'Coconut Yogurt Parfait with Berries',
 'Mozzarella Sticks',
 'Fresh Squeezed Orange Juice',
 'Homemade Veggie Pizza',
 'Caprese Skewers with Balsamic Glaze',
 'Caprese salad skewers',
 'Tomato and mozzarella skewers',
 'Stuffed dates with almond butter',
 'Greek Feta and Olive Bread',
 'Veggie Chips',
 'Mini Pizzas',
 'Yogurt Covered Strawberries',
 'Tomato and Mozzarella Skewers',
 'Caprese salad',
 'Turkish Gözleme',
 'Classical Arab Ful Medames',
 'Mini ham and cheese sandwiches',
 'Mini Pita Pizzas',
 'Fried Plantain',
 'Herb-roasted turkey breast with sweet potatoes',
 'Greek stuffed chicken breast with roasted potatoes',
 'Honey glazed sesame chicken with brown rice',
 'Cheese and Tomato Toastie',
 'Baked vegetable crisps',
 'Cherry tomato and mozzarella skewers',
 'Mini Bagel Chips with Smoked Salmon',
 'Roasted Pumpkin Seeds',
 'Yogurt Covered

In [112]:
df_recipes.loc[mask, 'taste'] = 'sweet'

## Continue fixing 

In [113]:
df = pd.read_csv('/home/victor/Documents/Expectation_data_generation/src/meals_collection/df_final_7000_with_ingredients_fixed.csv',
                 sep='|', index_col=0)

In [116]:
mask = df['preparation'].isna()
print(sum(mask))
df.loc[mask, 'preparation'] = ["""Slice beef thinly and marinate with soy sauce, garlic, ginger, and a bit of cornstarch.
Blanch broccoli florets in boiling water for a few minutes until slightly tender.
Stir-fry marinated beef in a hot wok until browned, then set aside.
In the same wok, stir-fry broccoli with additional garlic and ginger until vibrant green.
Return beef to the wok, add a sauce made of soy sauce, oyster sauce, and a bit of sugar.
Cook until the sauce thickens and coats the beef and broccoli. Serve hot with steamed rice.
                                       """, 
                                       """
                                       Marinate pork strips in a mixture of hoisin sauce, soy sauce, honey, Chinese five-spice powder, and garlic.
Let the pork marinate for a few hours or overnight.
Roast the marinated pork in the oven or grill until cooked through and slightly charred on the edges.
Baste with additional marinade during cooking for extra flavor.
Slice and serve hot, typically with steamed rice or noodles.
                                       """, 
                                       """
                                       Rinse rice and place it in a large pot with plenty of water.
Bring to a boil, then reduce heat to low and simmer, stirring occasionally.
As the rice cooks and breaks down, it will thicken into a porridge-like consistency.
Season with salt and pepper to taste.
Serve hot with various toppings such as green onions, shredded chicken, fried shallots, soy sauce, and pickled vegetables.
                                       """, 
                                       """
                                       Cook noodles according to package instructions and drain.
Meanwhile, prepare the sauce by mixing together soy sauce, sesame paste, chili oil, vinegar, sugar, and Sichuan peppercorn powder.
Cook ground pork with ginger, garlic, and Sichuan peppercorn until browned and fragrant.
Combine cooked noodles with the sauce and cooked pork, tossing well to coat.
Serve topped with chopped green onions and crushed peanuts for garnish.
                                       """,
                                       """
                                       Cook rice and let it cool completely, preferably using day-old rice.
Heat oil in a wok or large skillet over high heat.
Add beaten eggs and scramble until just set, then remove from the wok and set aside.
Add more oil to the wok and stir-fry finely chopped vegetables like carrots, peas, and bell peppers until tender.
Add the cooled rice to the wok along with soy sauce and any desired seasonings.
Stir-fry everything together until the rice is heated through and evenly coated with the sauce.
Add the scrambled eggs back into the wok and toss everything together until well combined.
Serve hot, garnished with chopped green onions.
                                       """, 
                                       """
                                       Prepare a broth by simmering chicken or vegetable stock with aromatics like ginger, garlic, and mushrooms.
Season the broth with soy sauce, vinegar, and a bit of sugar for balance.
Add in tofu, bamboo shoots, and any desired protein like shredded chicken or pork.
Thicken the soup with a slurry made of cornstarch and water.
Just before serving, swirl in beaten eggs to create ribbons.
Adjust seasoning with salt, pepper, and additional vinegar or soy sauce to taste.
Garnish with chopped green onions and a drizzle of sesame oil before serving.
                                       """,
                                       """
                                       Marinate chicken pieces in soy sauce, rice wine, and cornstarch for about 30 minutes.
Heat oil in a wok over high heat and stir-fry marinated chicken until cooked through.
Remove chicken from the wok and set aside.
In the same wok, stir-fry dried red chilies, Sichuan peppercorns, and peanuts until fragrant.
Add minced garlic, ginger, and diced bell peppers, stirring constantly.
Return the cooked chicken to the wok and add a sauce made of soy sauce, hoisin sauce, vinegar, and sugar.
Cook until the sauce thickens and coats the chicken and vegetables.
Serve hot with steamed rice.
                                       """, 
                                       """
                                       Sauté ground pork with garlic, ginger, and doubanjiang (fermented bean paste) until browned.
Add diced tofu and gently stir to combine.
Pour in a sauce made of soy sauce, fermented black beans, and chicken or vegetable stock.
Simmer until the tofu absorbs the flavors of the sauce.
Thicken the sauce with a slurry made of cornstarch and water.
Garnish with sliced green onions and a drizzle of sesame oil before serving.
                                       """,
                                       """
                                       Rub duck with a mixture of Chinese five-spice powder, salt, and sugar, and let it marinate overnight.
Roast duck in a hot oven until the skin is crispy and golden brown.
Serve sliced duck with thin pancakes, hoisin sauce, sliced cucumbers, and green onions.
To eat, spread hoisin sauce on a pancake, add slices of duck, cucumber, and green onions, then roll up and enjoy.
                                       """, 
                                       """
                                       Mix together flour, water, and salt to form a dough.
Roll out the dough into a thin rectangle on a floured surface.
Brush the dough with sesame oil and sprinkle chopped scallions evenly over the surface.
Roll up the dough into a tight cylinder, then coil it into a spiral.
Flatten the coil into a pancake shape.
Pan-fry the pancake in a hot skillet until golden brown and crispy on both sides.
Serve hot, sliced into wedges, with dipping sauce.
                                       """,
                                       """
                                       Blanch green beans in boiling water for a few minutes, then drain and set aside.
Heat oil in a wok and stir-fry minced garlic, ginger, and dried red chilies until fragrant.
Add blanched green beans to the wok and stir-fry until tender-crisp.
Season with soy sauce, sugar, and a splash of rice vinegar.
Toss everything together until the beans are evenly coated with the sauce.
Serve hot, garnished with chopped green onions and sesame seeds.
                                       """, 
                                       """
                                       Prepare a filling by combining ground meat (usually pork or chicken) with minced vegetables, garlic, ginger, soy sauce, sesame oil, and other seasonings.
Place a small amount of filling in the center of a dumpling wrapper.
Moisten the edges of the wrapper with water, then fold and pleat the edges to seal the dumpling.
Arrange the dumplings in a steamer basket and steam until cooked through, about 10-15 minutes.
Serve hot with soy sauce or dipping sauce of choice.
                                       """,
                                       """
                                       Grate turnips and mix them with rice flour, water, and seasonings like salt, white pepper, and dried shrimp.
Pour the mixture into a greased pan and steam until set and cooked through.
Let the turnip cake cool completely, then slice into pieces.
Pan-fry the sliced turnip cakes until crispy and golden brown on both sides.
Serve hot with soy sauce or hoisin sauce.
                                       """, 
                                       """
                                       Prepare a filling by mixing ground pork with minced garlic, ginger, green onions, soy sauce, sesame oil, and a bit of cornstarch.
Place a small amount of filling in the center of a wonton wrapper.
Moisten the edges of the wrapper with water, then fold and seal the wonton into desired shapes.
Bring chicken or vegetable broth to a simmer and carefully drop in the wontons.
Cook until the wontons float to the surface and the filling is cooked through.
Season the soup with salt, pepper, and additional soy sauce to taste.
Garnish with chopped green onions before serving.
                                       """,
                                       """
                                       Instructions:

Prepare the Noodles:

Cook the noodles according to package instructions until al dente.
Drain and rinse the noodles under cold water to stop the cooking process and prevent them from sticking together.
Toss the noodles with a bit of sesame oil to prevent sticking and set aside.
Prepare the Sauce:

In a small bowl, mix together 3-4 tablespoons of fermented yellow soybean paste (Huangjiang), 1 tablespoon of soy sauce, and 1 teaspoon of sugar. Adjust the quantities to your taste preference.
Stir until the soybean paste is well combined with the other ingredients. Set the sauce aside.
Cook the Ground Pork or Beef:

Heat a bit of cooking oil in a pan or wok over medium heat.
Add minced garlic and ginger to the pan and sauté until fragrant, about 1-2 minutes.
Add the ground pork or beef to the pan and cook, breaking it apart with a spatula, until browned and cooked through.
Once cooked, drain excess fat if necessary.
Combine Sauce and Meat:

Lower the heat and add the prepared sauce to the cooked meat in the pan.
Stir well to combine, ensuring that the meat is evenly coated with the sauce.
Allow the mixture to simmer for a few minutes to allow the flavors to meld together.
Assemble the Dish:

Place a portion of cooked noodles in serving bowls.
Spoon the meat and sauce mixture over the noodles.
Optionally, garnish with julienned cucumber, shredded carrots, and chopped green onions for added texture and freshness.
Serve:

Serve the Zhajiangmian hot, allowing each diner to mix the noodles and sauce together before eating.
You can also serve it with additional soy sauce or chili oil on the side for those who prefer extra flavor or spice.
Enjoy your homemade Zhajiangmian!
                                       """
                                       ] 

15


In [122]:
df.to_csv('/home/victor/Documents/Expectation_data_generation/src/meals_collection/df_final_7000_with_ingredients_prep_fixed.csv', 
          sep='|', encoding='utf-8')

In [117]:
mask = df['preparation'].isna()
print(sum(mask))

0
